In [12]:
import sentencepiece as spm
import re
from nltk import word_tokenize

#textfile = "inputText.txt"
textfile = "test.txt"

corpus = open (textfile,encoding="utf-8")

model = "model/sortie.txt"

content = corpus.readlines()
#tok_corp = content[0].split()

clean_content = re.sub('[^A-Za-z0-9]+', ' ', content[0])

print(clean_content)

#return a list
#tok_corp = word_tokenize(content[0])
tok_corp = word_tokenize(clean_content)

#get the tokenized text as string
toc = str(tok_corp).strip('[]')

#saving tokenized text
file = open('model/tokenized_corpus.txt', 'w')
file.write(toc)
file.close()

tokenized_corpus = "model/tokenized_corpus.txt"


spm.SentencePieceTrainer.train(input=tokenized_corpus, model_prefix = model, vocab_size = 32000, max_sentence_length=100000)

Thepurposeofourlivesistobehappy


sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: model/tokenized_corpus.txt
  input_format: 
  model_prefix: model/sortie.txt
  model_type: UNIGRAM
  vocab_size: 32000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 100000
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
}
normalizer_spec {
  name: nmt_nfkc
  add_dummy_prefix: 1
  remove_extra_

RuntimeError: Internal: src/trainer_interface.cc(590) [(trainer_spec_.vocab_size()) == (model_proto->pieces_size())] Vocabulary size too high (32000). Please set it to a value <= 22.

In [13]:
import re, collections

def get_vocab(filename):
    vocab = collections.defaultdict(int)
    with open(filename, 'r', encoding='utf-8') as fhand:
        for line in fhand:
            words = line.strip().split()
            for word in words:
                vocab[' '.join(list(word)) + ' </w>'] += 1

    return vocab

def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

def get_tokens_from_vocab(vocab):
    tokens_frequencies = collections.defaultdict(int)
    vocab_tokenization = {}
    for word, freq in vocab.items():
        word_tokens = word.split()
        for token in word_tokens:
            tokens_frequencies[token] += freq
        vocab_tokenization[''.join(word_tokens)] = word_tokens
    return tokens_frequencies, vocab_tokenization

def measure_token_length(token):
    if token[-4:] == '</w>':
        return len(token[:-4]) + 1
    else:
        return len(token)

def tokenize_word(string, sorted_tokens, unknown_token='</u>'):
    
    if string == '':
        return []
    if sorted_tokens == []:
        return [unknown_token]

    string_tokens = []
    for i in range(len(sorted_tokens)):
        token = sorted_tokens[i]
        token_reg = re.escape(token.replace('.', '[.]'))

        matched_positions = [(m.start(0), m.end(0)) for m in re.finditer(token_reg, string)]
        if len(matched_positions) == 0:
            continue
        substring_end_positions = [matched_position[0] for matched_position in matched_positions]

        substring_start_position = 0
        for substring_end_position in substring_end_positions:
            substring = string[substring_start_position:substring_end_position]
            string_tokens += tokenize_word(string=substring, sorted_tokens=sorted_tokens[i+1:], unknown_token=unknown_token)
            string_tokens += [token]
            substring_start_position = substring_end_position + len(token)
        remaining_substring = string[substring_start_position:]
        string_tokens += tokenize_word(string=remaining_substring, sorted_tokens=sorted_tokens[i+1:], unknown_token=unknown_token)
        break
    return string_tokens

# vocab = {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w e s t </w>': 6, 'w i d e s t </w>': 3}
#! wget http://www.gutenberg.org/cache/epub/16457/pg16457.txt
#vocab = get_vocab('pg16457.txt')
vocab = get_vocab('inputText.txt')

print('==========')
print('Tokens Before BPE')
tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(vocab)
print('All tokens: {}'.format(tokens_frequencies.keys()))
print('Number of tokens: {}'.format(len(tokens_frequencies.keys())))
print('==========')

num_merges = 100
for i in range(num_merges):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    print('Iter: {}'.format(i))
    print('Best pair: {}'.format(best))
    tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(vocab)
    print('All tokens: {}'.format(tokens_frequencies.keys()))
    print('Number of tokens: {}'.format(len(tokens_frequencies.keys())))
    print('==========')

# Let's check how tokenization will be for a known word
word_given_known = 'mountains</w>'
word_given_unknown = 'Ilikeeatingapples!</w>'

sorted_tokens_tuple = sorted(tokens_frequencies.items(), key=lambda item: (measure_token_length(item[0]), item[1]), reverse=True)
sorted_tokens = [token for (token, freq) in sorted_tokens_tuple]

print(sorted_tokens)

word_given = word_given_known 

print('Tokenizing word: {}...'.format(word_given))
if word_given in vocab_tokenization:
    print('Tokenization of the known word:')
    print(vocab_tokenization[word_given])
    print('Tokenization treating the known word as unknown:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))
else:
    print('Tokenizating of the unknown word:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))

word_given = word_given_unknown 

print('Tokenizing word: {}...'.format(word_given))
if word_given in vocab_tokenization:
    print('Tokenization of the known word:')
    print(vocab_tokenization[word_given])
    print('Tokenization treating the known word as unknown:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))
else:
    print('Tokenizating of the unknown word:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))


Tokens Before BPE
All tokens: dict_keys(['t', 'h', 'e', '</w>', 'i', 'n', 'r', 's', 'm', 'o', 'v', 'a', 'l', 'f', 'g', 'c', 'd', '.', 'p', 'y', 'u', '-', 'j', 'b', 'w', 'k', ',', 'z', '<', '>', '2', '/', 'x', '≡', ';', 'F', 'q', '(', 'U', 'P', 'S', ')', 'A', 'C', 'H', '5', '9', '4', '1', '3', '7', '6', 'O', 'B', 'R', 'N', 'M', 'I', 'T', 'W', '0', '·', '8', '%', 'D', 'E', 'G', "'", 'L', 'V', 'J', '∑', ':', 'K', 'μ', '×', '"', 'X', 'Y', 'Q', '=', '≥', 'α', '[', ']', '°', '{', '}', 'η', 'κ', '+', '®', '±', '≤', 'β', 'Z'])
Number of tokens: 96
Iter: 0
Best pair: ('e', '</w>')
All tokens: dict_keys(['t', 'h', 'e</w>', 'i', 'n', 'e', 'r', 's', '</w>', 'm', 'o', 'v', 'a', 'l', 'f', 'g', 'c', 'd', '.', 'p', 'y', 'u', '-', 'j', 'b', 'w', 'k', ',', 'z', '<', '>', '2', '/', 'x', '≡', ';', 'F', 'q', '(', 'U', 'P', 'S', ')', 'A', 'C', 'H', '5', '9', '4', '1', '3', '7', '6', 'O', 'B', 'R', 'N', 'M', 'I', 'T', 'W', '0', '·', '8', '%', 'D', 'E', 'G', "'", 'L', 'V', 'J', '∑', ':', 'K', 'μ', '×', '"', '

Iter: 22
Best pair: ('r', 'o')
All tokens: dict_keys(['the</w>', 'in', 't', 'er', 'e', 'st', '</w>', 're', 'm', 'o', 'v', 'al', 'f', 'g', 'c', 'on', 'a', 'an', 's</w>', 'h', 's', 'ed</w>', 'l', 'd', '.</w>', 'p', 'at', 'y', 'ti', 'r', 'u', '-', 'n', 'j', 'd</w>', 'y</w>', 'ro', 'i', 'e</w>', 'th', 'su', 'b', 'en', 'w', 'or', 'k', ',</w>', 'z', 'and</w>', '<', '>', '2', '/', 'x', '≡', ';', '.', 'F', 'q', '(', 'U', 'P', 'S', ')', 'A', 'C', 'H', ',', '5', '9', '4', '1', '3', '7', '6', 'O', 'B', 'R', 'N', 'M', 'I', 'T', 'W', '0', '·', '8', '%', 'D', 'E', 'G', "'", 'L', 'V', 'J', '∑', ':', 'K', 'μ', '×', '"', 'X', 'Y', 'Q', '=', '≥', 'α', '[', ']', '°', '{', '}', 'η', 'κ', '+', '®', '±', '≤', 'β', 'Z'])
Number of tokens: 119
Iter: 23
Best pair: ('in', '</w>')
All tokens: dict_keys(['the</w>', 'in', 't', 'er', 'e', 'st', '</w>', 'in</w>', 're', 'm', 'o', 'v', 'al', 'f', 'g', 'c', 'on', 'a', 'an', 's</w>', 'h', 's', 'ed</w>', 'l', 'd', '.</w>', 'p', 'at', 'y', 'ti', 'r', 'u', '-', 'n', 'j', '

Iter: 35
Best pair: ('in', 'g</w>')
All tokens: dict_keys(['the</w>', 'in', 't', 'er', 'e', 'st', '</w>', 'in</w>', 're', 'm', 'o', 'v', 'al', 'of</w>', 'g', 'ing</w>', 'c', 'on', 'a', 'an', 's</w>', 'h', 's', 'ed</w>', 'l', 'd', '.</w>', 'p', 'to', 'at', 'y', 'ti', 'r', 'tion', 'u', '-', 'n', 'j', 'd</w>', 'f', 'el', 'y</w>', 'ro', 'i', 'e</w>', 'th', 'su', 'b', 'ar', 'si', 'en', 't</w>', 'w', 'or', 'k', ',</w>', 'z', 'and</w>', '<', '>', '2', '</', 'f</w>', '/', 'x', 'ic', '≡', ';', '.', 'F', 'q', '(', 'U', 'P', 'S', ')', 'A', 'g</w>', 'C', 'H', ',', '5', '9', '4', '1', '3', '7', '6', 'O', 'B', 'R', 'N', 'M', 'I', 'T', 'W', '0', '·', '8', '%', 'D', 'E', 'G', "'", 'L', 'V', 'J', '∑', ':', 'K', 'μ', '×', '"', 'X', 'Y', 'Q', '=', '≥', 'α', '[', ']', '°', '{', '}', 'η', 'κ', '+', '®', '±', '≤', 'β', 'Z'])
Number of tokens: 132
Iter: 36
Best pair: ('d', 'i')
All tokens: dict_keys(['the</w>', 'in', 't', 'er', 'e', 'st', '</w>', 'in</w>', 're', 'm', 'o', 'v', 'al', 'of</w>', 'g', 'ing</w>',

All tokens: dict_keys(['the</w>', 'in', 't', 'er', 'e', 'st', '</w>', 'in</w>', 're', 'm', 'o', 'v', 'al</w>', 'of</w>', 'g', 'ing</w>', 'c', 'on', 'a', 'an', 's</w>', 'h', 's', 'ed</w>', 'l', 'd', 'ec', '.</w>', 'p', 'to', 'at', 'al', 'y', 'ti', 'ra', 'tion', 'u', '-', 'n', 'j', 'co', 'd</w>', 'f', 'el', 'y</w>', 'ro', 'i', 'e</w>', 'th', 'sub', 'es</w>', 'ar', 'si', 'en', 't</w>', 'r', 'w', 'or', 'k', ',</w>', 'z', 'and</w>', '<', 'sub>', '2', '</', 'f</w>', '/', 'di', 'x', 'ic', 'b', 'a</w>', 'su', 'it', '≡', ';', '.', 'F', 'q', '(', 'U', 'P', 'S', ')', 'A', 'g</w>', 'C', 'H', ',', '5', '9', '4', '1', '3', '7', '6', 'O', 'B', 'R', 'N', 'M', 'I', 'T', 'W', '0', '·', '>', '8', '%', 'D', 'E', 'G', "'", 'L', 'V', 'J', '∑', ':', 'K', 'μ', '×', '"', 'i>', 'X', 'Y', 'Q', '=', '≥', 'α', '[', ']', '°', '{', '}', 'η', 'κ', '+', '®', '±', '≤', 'β', 'Z'])
Number of tokens: 143
Iter: 47
Best pair: ('tion', '</w>')
All tokens: dict_keys(['the</w>', 'in', 't', 'er', 'e', 'st', '</w>', 'in</w>', 'r

Iter: 58
Best pair: ('a', 's')
All tokens: dict_keys(['the</w>', 'in', 't', 'er', 'e', 'st', '</w>', 'in</w>', 're', 'm', 'o', 'v', 'al</w>', 'of</w>', 'g', 'ing</w>', 'c', 'on', 'a', 'an', 's</w>', 'h', 'as</w>', 'as', 'ed</w>', 'l', 'd', 'ec', '.</w>', 'p', 'to', 'at', 'al', 'y', 'ti', 'ra', 'tion</w>', 'u', 's', '-', 'n', 'j', 'tion', 'co', 'd</w>', 'f', 'el', 'y</w>', 'pro', 'i', 'e</w>', 'th', 'es', 'sub', 'es</w>', 'ar', 'si', 'en', 't</w>', 'r', 'w', 'or', 'k', ',</w>', 'z', 'and</w>', '<', 'sub>', '2', '</', 'f</w>', '/', 'di', 'x', 'ic', 'b', 'to</w>', 'a</w>', 'su', 'it', '≡', 'sp', 'or</w>', 'th</w>', ';', '.', 'F-', 'q', '(', 'U', 'P', 'S', ')', 'ro', 'A', 'g</w>', 'C', 'er</w>', 'H', ',', '5', '9', '4', '1', '3', '7', '6', 'O', 'B', 'R', 'N', 'M', 'I', 'T', 'W', 'F', '0', '·', 'sup', '>', '8', '%', 'D', 'E', 'G', "'", 'L', 'V', 'J', '∑', ':', 'K', 'μ', '×', '"', 'i>', 'X', 'Y', 'Q', '=', '≥', 'α', '[', ']', '°', '{', '}', 'η', 'κ', '+', '®', '±', '≤', 'β', 'Z'])
Number of 

Iter: 68
Best pair: ('u', 'n')
All tokens: dict_keys(['the</w>', 'in', 't', 'er', 'e', 'st', '</w>', 'in</w>', 're', 'm', 'o', 'v', 'al</w>', 'of</w>', 'g', 'ing</w>', 'c', 'on', 'a', 'an', 's</w>', 'h', 'as</w>', 'as', 'ed</w>', 'l', 'd', 'ec', '.</w>', 'p', 'to', 'at', 'al', 'y', 'ti', 'ra', 'tion</w>', 'u', 's', '-', 'n', 'et', 'j', 'un', 'tion', 'co', 'ul', 'd</w>', 'f', 'tiv', 'el', 'y</w>', 'pro', 'i', 'e</w>', 'th', 'es', 'sub', 'es</w>', 'ar', 'si', 'en', 't</w>', 'r', 'w', 'or', 'k', ',</w>', 'ac', 'z', 'and</w>', '<', 'sub>', '2', '</', 'f</w>', '/', 'ere</w>', 'di', 'x', 'ic', 'b', 'to</w>', 'a</w>', 'su', 'it', '≡', 'sp', 'or</w>', 'wi', 'th</w>', ';', '.', 'F-', 'q', '(', 'U', 'P', 'S', ')</w>', 'ro', 'A', 'g</w>', 'C', 'er</w>', 'H', ',', '5', '9', '4', '1', '3', '7', '6', 'O', 'B', 'R', 'N', 'M', 'I', 'T', 'W', ')', 'F', '0', '·', 'sup>', '8', '%', 'F</w>', 'D', 'E', 'G', "'", 'sup', 'L', 'V', 'J', '∑', ':', 'K', 'μ', '×', '"', 'i>', 'X', 'Y', 'Q', '=', '≥', 'α', '[', ']

All tokens: dict_keys(['the</w>', 'in', 't', 'er', 'e', 'st', '</w>', 'in</w>', 're', 'm', 'o', 'v', 'al</w>', 'of</w>', 'g', 'ing</w>', 'c', 'on', 'a', 'an', 's</w>', 'h', 'as</w>', 'as', 'ed</w>', 'l', 'd', 'ec', '.</w>', 'p', 'to', 'at', 'al', 'y', 'tic', 'ra', 'ation</w>', 'us', '-', 'n', 'et', 'j', 'un', 'tion', 'co', 'ul', 'd</w>', 'f', 'tiv', 'el', 'y</w>', 'pro', 'i', 'e</w>', 'th', 'es', 'sub', 'es</w>', 'ar', 'si', 'en', 't</w>', 'r', 'ent', 'w', 'or', 'k', ',</w>', 's', 'ac', 'z', 'and</w>', 'u', 'ti', '<', 'sub>', '2', '</', 'f</w>', '/', 'mp', 'were</w>', 'di', 'm</w>', 'x', 'ic', 'ri', 'b', 'to</w>', 'a</w>', 'su', 'it', '≡', 'sp', 'or</w>', 'with</w>', 'tion</w>', ';', '.', 'F-', 'q', '(', 'U', 'P', 'S', ')</w>', 'ro', 'A', 'g</w>', 'C', 'er</w>', 'ol', 'H', ',', '5', '9', '4', '1', '3', '7', '6', 'O', 'ur', 'B', 'R', 'N', 'M', 'th</w>', 'wi', 'I', 'Th', 'W', 'T', ')', 'F', '0', '·', 'sup>', '8', '%', 'F</w>', 'D', 'E', 'G', "'", 'sup', 'L', 'V', 'J', '∑', ':', 'K', 'μ',

Iter: 91
Best pair: ('C', '-')
All tokens: dict_keys(['the</w>', 'in', 't', 'er', 'e', 'st', '</w>', 'in</w>', 're', 'm', 'o', 'v', 'al</w>', 'of</w>', 'g', 'ing</w>', 'c', 'on', 'a', 'an', 's</w>', 'h', 'as</w>', 'as', 'ed</w>', 'l', 'd', 'ec', '.</w>', 'p', 'to', 'at', 'al', 'y', 'tic', 'ra', 'ation</w>', 'us', '-', 'n', 'et', 'j', 'un', 'tion', 'co', 'ul', 'd</w>', 'f', 'tiv', 'el', 'y</w>', 'pro', 'i', 'e</w>', 'th', 'es', 'sub', 'es</w>', 'ar', 'si', 'en', 't</w>', 'ir', 'ent', 'w', 'or', 'k', ',</w>', 's', 'ch', 'ac', 'z', 'and</w>', 'ev', 'u', 'ti', '<', 'sub>', '2', '</', 'f</w>', 'l</w>', '/', 'mp', 'were</w>', 'di', 'm</w>', 'x', 'ic', 'ri', 'b', 'to</w>', 'a</w>', 'su', 'r', 'it', '≡', 'sp', 'or</w>', 'is', 'on</w>', 'with</w>', 'tion</w>', ';', '.', 'F-', 'q', '(', 'U', 'P', 'S', ')</w>', 'ro', 'A', 'g</w>', 'C', 'ex', 'er</w>', 'ol', 'H', ',', '5', '9', '4', '1', '3', '7', '6', 'O', 'ur', 'B', 'R', 'N', 'M', 'th</w>', 'wi', 'I', 'Th', 'W', 'T', ')', 'F', '0.', '·', 'sup>',

In [17]:
import requests
import xlsxwriter
from bs4 import BeautifulSoup


URL = 'https://pubmed.ncbi.nlm.nih.gov/?term=COVID+19&size=200'
response = requests.get(URL)

print(response)

html_soup = BeautifulSoup(response.text, 'html5lib')
article_containers = html_soup.find_all('article', class_ = 'labs-full-docsum')
print(article_containers)

first_article = article_containers[0]
citation_text = first_article.find('div', class_ = 'docsum-wrap').find('div', class_ = 'result-actions-bar').div.div.find('div', class_ = 'content').div.div.text

print(citation_text)

<Response [200]>
[]


IndexError: list index out of range

In [11]:
#!pip install pymed
from pymed import PubMed
import pandas as pd
pubmed = PubMed(tool="PubMedSearcher", email="myemail@ccc.com")

## PUT YOUR SEARCH TERM HERE ##
search_term = "fever"
TERM = 'for DM it was 0.707, for organic matter (OM) it was 0.724, for CP it was 0.861, and for NDF it was 0.609. In comparison with alfalfa silage, Virginia fanpetals silage was characterized by higher apparent digestibility of nutrients, but a significant difference was noted only for CP. The voluntary intake of Virginia fanpetals silage was significantly higher than that of alfalfa silage (1427.4 vs. 954g DM). The greatest differences in voluntary intake were observed 0-2 and 8-12h after feeding. Virginia fanpetals silage had a chemical composition similar to that of alfalfa, but it was characterized by a more desirable fermentation pattern and higher digestibility, and it was more willingly consumed by rams'

results = pubmed.query(TERM, max_results=5000000000)
articleList = []
articleInfo = []

for article in results:
# Print the type of object we've found (can be either PubMedBookArticle or PubMedArticle).
# We need to convert it to dictionary with available function
    articleDict = article.toDict()
    articleList.append(articleDict)

# Generate list of dict records which will hold all article details that could be fetch from PUBMED API
for article in articleList:
#Sometimes article['pubmed_id'] contains list separated with comma - take first pubmedId in that list - thats article pubmedId
    pubmedId = article['pubmed_id'].partition('\n')[0]
    # Append article info to dictionary 
    articleInfo.append({u'pubmed_id':pubmedId,
                       #u'title':article['title'],
                       #u'keywords':article['keywords'],
                       #u'journal':article['journal'],
                       u'abstract':article['abstract'],
                       u'conclusions':article['conclusions'],
                       #u'methods':article['methods'],
                       u'results': article['results'],
                       #u'copyrights':article['copyrights'],
                       #u'doi':article['doi'],
                       #u'publication_date':article['publication_date'], 
                       #u'authors':article['authors']
                       })
print(articleInfo)
# Generate Pandas DataFrame from list of dictionaries
#articlesPD = pd.DataFrame.from_dict(articleInfo)
#export_csv = df.to_csv (r'C:\Users\YourUsernam\Desktop\export_dataframe.csv', index = None, header=True) 

#Print first 10 rows of dataframe
#print(articlesPD.head(10))

[{'pubmed_id': '35252546', 'abstract': 'The aim of the current study is to evaluate Virginia fanpetals silage based on an apparent digestibility and palatability test performed on six adult rams. Alfalfa silage was used as standard forage for comparison. Virginia fanpetals samples were harvested in the bud-formation stage and alfalfa samples were harvested in the late bud stage. Virginia fanpetals silage had a crude protein (CP) content of 176\u202fg\u202fkg ', 'conclusions': None, 'results': None}]


In [10]:
#!/usr/bin/env python
# numpy and biopython are required -- pip install numpy biopython

from Bio import Entrez
from Bio import Medline

MAX_COUNT = 100
TERM = 'for DM it was 0.707, for organic matter (OM) it was 0.724, for CP it was 0.861, and for NDF it was 0.609. In comparison with alfalfa silage, Virginia fanpetals silage was characterized by higher apparent digestibility of nutrients, but a significant difference was noted only for CP. The voluntary intake of Virginia fanpetals silage was significantly higher than that of alfalfa silage (1427.4 vs. 954g DM). The greatest differences in voluntary intake were observed 0-2 and 8-12h after feeding. Virginia fanpetals silage had a chemical composition similar to that of alfalfa, but it was characterized by a more desirable fermentation pattern and higher digestibility, and it was more willingly consumed by rams'

print('Getting {0} publications containing {1}...'.format(MAX_COUNT, TERM))
Entrez.email = 'A.N.Other@example.com'
h = Entrez.esearch(db='pubmed', retmax=MAX_COUNT, term=TERM)
result = Entrez.read(h)
print('Total number of publications containing {0}: {1}'.format(TERM, result['Count']))
ids = result['IdList']
h = Entrez.efetch(db='pubmed', id=ids, rettype='medline', retmode='text')
records = Medline.parse(h)

authors = []
for record in records:
    au = record.get('AB', '?')
    #for a in au: 
        #if a not in authors:
    authors.append(au)
    authors.sort()
print('Authors: {0}'.format(', '.join(authors)))
    #authors.append(record)
    
#print(authors)

Getting 100 publications containing for DM it was 0.707, for organic matter (OM) it was 0.724, for CP it was 0.861, and for NDF it was 0.609. In comparison with alfalfa silage, Virginia fanpetals silage was characterized by higher apparent digestibility of nutrients, but a significant difference was noted only for CP. The voluntary intake of Virginia fanpetals silage was significantly higher than that of alfalfa silage (1427.4 vs. 954g DM). The greatest differences in voluntary intake were observed 0-2 and 8-12h after feeding. Virginia fanpetals silage had a chemical composition similar to that of alfalfa, but it was characterized by a more desirable fermentation pattern and higher digestibility, and it was more willingly consumed by rams...
Total number of publications containing for DM it was 0.707, for organic matter (OM) it was 0.724, for CP it was 0.861, and for NDF it was 0.609. In comparison with alfalfa silage, Virginia fanpetals silage was characterized by higher apparent dige